In [172]:
from datetime import datetime as dt
import datetime
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, WEEKLY
import requests
import pandas as pd
import numpy as np
import time
import pytz
from ta.trend import PSARIndicator

def unix_to_local_time(unix_timestamp):
    return str(dt.fromtimestamp(unix_timestamp).strftime('%Y-%m-%d %H:%M:%S'))

def send_telegram_message(message):
    TOKEN = "7045906590:AAGK5-gfPyqXXFx1cZJgfl9xUXcXcTBiUsI"
    chat_id = "1000631227"
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
    requests.get(url)

def get_expiry_day_of_week(day_of_week):
    """Return the date for a specified day of the week within the current week."""
    today = datetime.date.today()
    rule = rrule(WEEKLY, dtstart=today, byweekday=day_of_week, count=1)
    return rule[0].date()

def create_option_symbol(index_name, option_type, strike_price):
    """Generate an option symbol for NIFTY, BANKNIFTY, or MIDCPNIFTY with the current week's expiration date."""
    if option_type not in ["PE", "CE"]:
        raise ValueError("Invalid option type. Use 'PE' for Put or 'CE' for Call.")

    if index_name == "NIFTY":
        expiry_date = get_expiry_day_of_week(3)  # Thursday (2: Monday, 3: Thursday)
    elif index_name == "BANKNIFTY":
        expiry_date = get_expiry_day_of_week(2)  # Wednesday
    elif index_name == "MIDCPNIFTY":
        expiry_date = get_expiry_day_of_week(0)  # Monday
    else:
        raise ValueError("Invalid index name. Use 'NIFTY', 'BANKNIFTY', or 'MIDCPNIFTY'.")

    year = expiry_date.year % 100  # Last two digits of the year
    month = expiry_date.month  # Month as an integer
    day = expiry_date.day  # Day as an integer

    expiry_str = f"{year:02d}{month}{day:02d}"
    strike_price_str = f"{strike_price:05d}"
    option_symbol = f"{index_name}{expiry_str}{strike_price_str}{option_type}"
    return option_symbol

def get_ohlc_data(symbol,isOptionChart, price, ott, otype):
    if isOptionChart:
        daysToOHLC = 5
    else:
        daysToOHLC = 40
    end_time_in_millis = int(time.time() * 1000)
    end_time = datetime.datetime.fromtimestamp(end_time_in_millis / 1000)
    start_time = end_time - datetime.timedelta(days= daysToOHLC) #if fno, current week strike - 21, next week strike - 14, similarly use days
    start_time = start_time.replace(hour=9, minute=0, second=0, microsecond=0)
    start_time_in_millis = int(start_time.timestamp() * 1000)
    
    if isOptionChart:
        optionType = otype
        strikePrice = int(price)
        optionName = create_option_symbol(symbol, optionType, strikePrice)
        url = 'https://groww.in/v1/api/stocks_fo_data/v3/charting_service/chart/exchange/NSE/segment/FNO/'+optionName
        timeframe = ott
    else:
        url = 'https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/'+symbol
        timeframe = 240
        
    params = {
        'endTimeInMillis': str(end_time_in_millis),
        'intervalInMinutes': str(timeframe),
        'startTimeInMillis': str(start_time_in_millis),
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        df = pd.DataFrame(json_data)
    
    # Drop unwanted columns
    df.drop(['changeValue', 'changePerc', 'closingPrice', 'startTimeEpochInMillis'], axis=1, inplace=True)
    ohlc = df['candles'].apply(pd.Series)
    
    # Rename the columns to the desired format
    ohlc.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    ohlc.drop(['volume'], axis=1, inplace=True)
    
    if isOptionChart:
        # Calculate PSAR values
        psar = PSARIndicator(ohlc['high'], ohlc['low'], ohlc['close'], step=0.2)
        ohlc['psar'] = psar.psar()
    return ohlc

def get_trend(symbol):
    gap_ema_close = 30
    ohlc_data = get_ohlc_data(symbol, False)
    ohlc_data['EMA_9'] = ohlc_data['close'].ewm(span=9, adjust=False).mean()
    
    last_index = len(ohlc_data)-1
            
    current_close = ohlc_data.iloc[last_index]['close']
    current_ema_9 = ohlc_data.iloc[last_index]['EMA_9']
    print(current_close,current_ema_9)
    del ohlc_data
    trend = "PE"
    if current_close > current_ema_9 and abs(current_close - current_ema_9) >= gap_ema_close:
        trend = "CE"
        return trend
    elif current_close < current_ema_9 and abs(current_close - current_ema_9) >= gap_ema_close:
        trend = "PE"
        return trend
    else:
        return trend

In [185]:
symbol = 'NIFTY'
isOptionChart = True
price = 22000
ott = 15
otype = 'PE'
ohlc = get_ohlc_data(symbol, isOptionChart, price, ott,otype)

# ohlc_trend = get_trend(symbol)
# print(ohlc_trend)
# for index, df in ohlc_trend.iterrows():
#     if df['Trend'] == 'CE':
#         print(f"{unix_to_local_time(df['timestamp'])}, 'Uptrend', {df['close']},{df['EMA_9']}")
#     elif df['Trend'] == 'PE':
#         print(f"{unix_to_local_time(df['timestamp'])}, 'Downtrend', {df['close']},{df['EMA_9']}")


In [186]:
position = False
entry_price = None
trades = []
open_trades = []

# Assume 'ohlc' is your OHLC data DataFrame
ohlc['HA_Close'] = (ohlc['open'] + ohlc['high'] + ohlc['low'] + ohlc['close']) / 4
ha_open = ohlc['HA_Close'].shift(1)
ohlc['HA_Open'] = ha_open.values[0]
ohlc.loc[1:, 'HA_Open'] = ha_open.values[1:]
ohlc['HA_High'] = ohlc[['HA_Open', 'HA_Close', 'high']].max(axis=1)
ohlc['HA_Low'] = ohlc[['HA_Open', 'HA_Close', 'low']].min(axis=1)

# Variables to track trade
is_in_trade = False
entry_time = None

for index, row in ohlc.iterrows():
    # Assume 'df' is your OHLC data DataFrame
    df = ohlc

    if index == 0:
        i = 1
    else:
        i = index

    if row['HA_Close'] >= row['HA_Open']:  # Green candle
        if ohlc.iloc[i-1]['HA_Close'] >= ohlc.iloc[i-1]['HA_Open']:  # Previous candle was green
            if not is_in_trade:
                position = True
                entry_price = row['open']  # Use real candle open price for entry
                entry_time = row['timestamp']
                is_in_trade = True
                if sendMessage:
                    send_telegram_message(f"{optionName}: Entry: {entry_price}")
                trades.append(f"Buy: {unix_to_local_time(row['timestamp'])}, Price: {entry_price}")
                open_trades.append({'type': 'long', 'entry_price': entry_price, 'entry_time': row['timestamp']})

    else:  # Red candle
        if ohlc.iloc[i-1]['HA_Close'] < ohlc.iloc[i-1]['HA_Open']:  # Previous candle was red
            if position:
                position = False
                exit_price = row['open']  # Use real candle open price for exit
                exit_time = row['timestamp']
                if sendMessage:
                    send_telegram_message(f"{optionName}: Exit: {exit_price}, PL: {round(exit_price - entry_price)}")
                trade_details = {'type': 'long', 'entry_price': entry_price, 'exit_price': exit_price,
                                 'entry_time': unix_to_local_time(open_trades[-1]['entry_time']),
                                 'exit_time': unix_to_local_time(row['timestamp'])}
                open_trades.pop()
                trades.append(f"Exit: {unix_to_local_time(row['timestamp'])}, Price: {exit_price}, Profit/Loss: {exit_price - entry_price}")
                trades.append(trade_details)
                is_in_trade = False
                entry_price = 0
                entry_time = None

# Print trades taken
print("Trades Taken:")
for trade in trades:
    if isinstance(trade, dict):
        print(f"Entry: {trade['entry_time']} at {trade['entry_price']}, Exit: {trade['exit_time']} at {trade['exit_price']}, Profit/Loss: {trade['exit_price'] - trade['entry_price']}")

# Calculate total profit/loss
total_profit_loss = 0
for trade in trades:
    if isinstance(trade, dict):
        if trade['type'] == 'long':
            profit_loss = trade['exit_price'] - trade['entry_price']
            total_profit_loss += profit_loss

print(f"Overall Profit/Loss: {total_profit_loss}")

# Print open trades
if open_trades:
    print("\nOpen Trades:")
    for trade in open_trades:
        print(f"Type: {trade['type']}, Entry: {unix_to_local_time(trade['entry_time'])} at {trade['entry_price']}")

Trades Taken:
Entry: 2024-05-07 09:45:00 at 54.85, Exit: 2024-05-07 13:30:00 at 84.5, Profit/Loss: 29.65
Entry: 2024-05-07 14:30:00 at 90.65, Exit: 2024-05-08 09:45:00 at 81.0, Profit/Loss: -9.650000000000006
Entry: 2024-05-08 10:30:00 at 93.65, Exit: 2024-05-08 11:00:00 at 86.0, Profit/Loss: -7.650000000000006
Entry: 2024-05-08 13:45:00 at 60.6, Exit: 2024-05-08 14:45:00 at 60.05, Profit/Loss: -0.5500000000000043
Entry: 2024-05-09 09:30:00 at 79.6, Exit: 2024-05-09 11:00:00 at 98.1, Profit/Loss: 18.5
Entry: 2024-05-09 12:45:00 at 101.65, Exit: 2024-05-09 13:45:00 at 128.25, Profit/Loss: 26.599999999999994
Entry: 2024-05-09 14:45:00 at 154.0, Exit: 2024-05-10 09:30:00 at 132.0, Profit/Loss: -22.0
Entry: 2024-05-10 10:30:00 at 108.3, Exit: 2024-05-10 13:30:00 at 139.7, Profit/Loss: 31.39999999999999
Entry: 2024-05-10 14:15:00 at 133.7, Exit: 2024-05-10 15:00:00 at 128.15, Profit/Loss: -5.549999999999983
Overall Profit/Loss: 60.749999999999986


In [170]:
# <SUITABLE FOR HIGHER TIME FRAME 30M-60M - MIN DAYS 7, IF LOWER TIME FRAME MORE SINGALS WITH LESS PROFIT>
sendMessage = False

# Calculate PSAR values
# psar = PSARIndicator(ohlc['high'], ohlc['low'], ohlc['close'])
# ohlc['psar'] = psar.psar()

# Initialize variables
position = False 
entry_price = None
trades = []
open_trades = []

for index, row in ohlc.iterrows():
    # Check for buy signal
    if index == 0:
        i = 1
    else:
        i = index
        
    if row['open'] > row['psar'] and position != True:
        position = True
        entry_price = row['open']
        if sendMessage == True:
            send_telegram_message(f"{optionName}: Entry: {entry_price}")
        trades.append(f"Buy: {unix_to_local_time(row['timestamp'])}, Price: {entry_price}")
        open_trades.append({'type': 'long', 'entry_price': entry_price, 'entry_time': row['timestamp']})

    # Check for sell signal or red candle
    elif (row['open'] < row['psar'] or ohlc.iloc[i-1]['close'] < ohlc.iloc[i-1]['open']) and position == True:
    #elif (row['open'] < row['psar'] or row['close'] < row['open']) and position == True:
        position = False
        exit_price = row['open']
        if sendMessage == True:
            send_telegram_message(f"{optionName}: Exit: {exit_price}, PL: {round(exit_price - entry_price)}")
        trade_details = {'type': 'long', 'entry_price': entry_price, 'exit_price': exit_price,
                         'entry_time': unix_to_local_time(open_trades[-1]['entry_time']),
                         'exit_time': unix_to_local_time(row['timestamp'])}
        open_trades.pop()
        trades.append(f"Exit: {unix_to_local_time(row['timestamp'])}, Price: {exit_price}, Profit/Loss: {exit_price - entry_price}")
        trades.append(trade_details)

# Print trades taken
print("Trades Taken:")
for trade in trades:
    if isinstance(trade, dict):
        print(f"Entry: {trade['entry_time']} at {trade['entry_price']}, Exit: {trade['exit_time']} at {trade['exit_price']}, Profit/Loss: {trade['exit_price'] - trade['entry_price']}")
    # else:
    #     print(trade)

total_profit_loss = 0
for trade in trades:
    if isinstance(trade, dict):
        if trade['type'] == 'long':
            profit_loss = trade['exit_price'] - trade['entry_price']
            total_profit_loss += profit_loss

print(f"Overall Profit/Loss: {total_profit_loss}")

# Print open trades
if open_trades:
    print("\nOpen Trades:")
    for trade in open_trades:
        print(f"Type: {trade['type']}, Entry: {unix_to_local_time(trade['entry_time'])} at {trade['entry_price']}")

#print(ohlc)

Trades Taken:
Entry: 2024-05-07 09:15:00 at 55.0, Exit: 2024-05-07 09:30:00 at 51.9, Profit/Loss: -3.1000000000000014
Entry: 2024-05-07 10:00:00 at 63.1, Exit: 2024-05-07 11:00:00 at 74.0, Profit/Loss: 10.899999999999999
Entry: 2024-05-07 11:15:00 at 84.15, Exit: 2024-05-07 11:45:00 at 92.55, Profit/Loss: 8.399999999999991
Entry: 2024-05-07 12:00:00 at 92.5, Exit: 2024-05-07 12:15:00 at 91.6, Profit/Loss: -0.9000000000000057
Entry: 2024-05-07 12:45:00 at 98.95, Exit: 2024-05-07 13:00:00 at 96.5, Profit/Loss: -2.450000000000003
Entry: 2024-05-07 14:30:00 at 90.65, Exit: 2024-05-07 14:45:00 at 74.8, Profit/Loss: -15.850000000000009
Entry: 2024-05-08 09:15:00 at 94.45, Exit: 2024-05-08 09:30:00 at 91.75, Profit/Loss: -2.700000000000003
Entry: 2024-05-08 10:15:00 at 71.8, Exit: 2024-05-08 10:45:00 at 92.6, Profit/Loss: 20.799999999999997
Entry: 2024-05-08 11:00:00 at 86.0, Exit: 2024-05-08 11:15:00 at 81.6, Profit/Loss: -4.400000000000006
Entry: 2024-05-08 13:30:00 at 49.65, Exit: 2024-05-